In [1]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from matplotlib import pyplot as plt
from joblib import dump, load

In [2]:
import pandas as pd
df = pd.read_csv('flight_data.csv', on_bad_lines='skip')
df.head(5)

,Unnamed: 0,from_airport_code,from_country,dest_airport_code,dest_country,aircraft_type,airline_number,airline_name,flight_number,departure_time,arrival_time,duration,stops,price,currency,co2_emissions,avg_co2_emission_for_this_route,co2_percentage,scan_date
0,206398,BRU,Belgium,MAD,Spain,Airbus A319|ATR 72|Airbus A319,multi,[Brussels Airlines| Tap Air Portugal],SN3809|SN3809|SN3809,2022-08-27 14:40:00,2022-08-28 00:20:00,580,2,317.0,USD,338000.0,149000.0,127%,2022-04-29 17:52:59
1,464767,XIY,China,CGO,China,Airbus A321|Boeing 737,multi,[China Eastern| Shanghai Airlines],MU2155|MU2155,2022-05-15 11:00:00,2022-05-15 16:30:00,330,1,636.0,USD,284000.0,NaN,NaN,2022-04-29 17:52:59
2,825242,MUC,Germany,YYZ,Canada,Airbus A320|Embraer 195|Boeing 787,multi,[Lufthansa| LOT],LH1622|LH1622|LH1622,2022-05-30 11:20:00,2022-05-30 20:15:00,895,2,903.0,USD,684000.0,656000.0,4%,2022-04-29 17:52:59
3,510029,HGH,China,SEA,United States,Airbus A321neo|Airbus A350|Boeing 737|De Havil...,multi,[Cathay Pacific| Alaska],CX959|CX959|CX959|CX959,2022-05-30 11:25:00,2022-05-31 20:29:00,2884,3,2586.0,USD,1133000.0,1119000.0,1%,2022-04-29 17:52:59
4,998709,BOM,India,BKK,Thailand,Airbus A320|Airbus A320,6E,[IndiGo],6E2518|6E2518,2022-05-07 20:35:00,2022-05-08 11:20:00,795,1,229.0,USD,412000.0,340000.0,20%,2022-04-29 17:52:59


In [3]:
# Drop duplicate values
df.drop_duplicates(inplace=True)

# Drop rows with missing values
df.dropna(inplace=True)

In [4]:
# remove the airline_name column in  pandas DataFrame that contains square brackets []

df['airline_name'] = df['airline_name'].str.replace('[','').str.replace(']','')
df['airline_name'] = df['airline_name'].str.split('|').str[0]

/var/folders/t3/ctmyp9cd5j7ctjhqp2_g843h0000gn/T/ipykernel_56736/555518936.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['airline_name'] = df['airline_name'].str.replace('[','').str.replace(']','')


In [5]:
# delete the duplicate flight number in column flight_number behind the strings with a | separator

df['flight_number'] = df['flight_number'].str.split('|').str[0]

In [6]:
# Get the datetime info

df['departure_month'] = pd.to_datetime(df['departure_time']).dt.month

In [7]:
# Check Whether the units are standardized

df['currency'].unique()

array(['USD'], dtype=object)

In [8]:
df['co2_percentage'] = df['co2_percentage'].str.replace('%','')
df.head()

,Unnamed: 0,from_airport_code,from_country,dest_airport_code,dest_country,aircraft_type,airline_number,airline_name,flight_number,departure_time,arrival_time,duration,stops,price,currency,co2_emissions,avg_co2_emission_for_this_route,co2_percentage,scan_date,departure_month
0,206398,BRU,Belgium,MAD,Spain,Airbus A319|ATR 72|Airbus A319,multi,Brussels Airlines,SN3809,2022-08-27 14:40:00,2022-08-28 00:20:00,580,2,317.0,USD,338000.0,149000.0,127,2022-04-29 17:52:59,8
2,825242,MUC,Germany,YYZ,Canada,Airbus A320|Embraer 195|Boeing 787,multi,Lufthansa,LH1622,2022-05-30 11:20:00,2022-05-30 20:15:00,895,2,903.0,USD,684000.0,656000.0,4,2022-04-29 17:52:59,5
3,510029,HGH,China,SEA,United States,Airbus A321neo|Airbus A350|Boeing 737|De Havil...,multi,Cathay Pacific,CX959,2022-05-30 11:25:00,2022-05-31 20:29:00,2884,3,2586.0,USD,1133000.0,1119000.0,1,2022-04-29 17:52:59,5
4,998709,BOM,India,BKK,Thailand,Airbus A320|Airbus A320,6E,IndiGo,6E2518,2022-05-07 20:35:00,2022-05-08 11:20:00,795,1,229.0,USD,412000.0,340000.0,20,2022-04-29 17:52:59,5
6,329433,YYZ,Canada,HND,Japan,Embraer 175|Boeing 777,multi,American,AA4827,2022-05-02 06:45:00,2022-05-03 16:35:00,1250,1,956.0,USD,1367000.0,1250000.0,9,2022-04-29 17:52:59,5


In [9]:
# Change object variables to category type

for col in df.columns:
    # Check if the column is of object type
    if df[col].dtype == 'object':
        # If it is, convert it to category type
        df[col] = df[col].astype('category')

In [10]:
df['co2_percentage'] = df['co2_percentage'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44700 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   Unnamed: 0                       44700 non-null  int64   
 1   from_airport_code                44700 non-null  category
 2   from_country                     44700 non-null  category
 3   dest_airport_code                44700 non-null  category
 4   dest_country                     44700 non-null  category
 5   aircraft_type                    44700 non-null  category
 6   airline_number                   44700 non-null  category
 7   airline_name                     44700 non-null  category
 8   flight_number                    44700 non-null  category
 9   departure_time                   44700 non-null  category
 10  arrival_time                     44700 non-null  category
 11  duration                         44700 non-null  int64   
 12  stop

In [11]:
from_country_mapping = dict(enumerate(df['from_country'].cat.categories))
print(from_country_mapping)

{0: 'Algeria', 1: 'Argentina', 2: 'Australia', 3: 'Austria', 4: 'Belgium', 5: 'Brazil', 6: 'Canada', 7: 'Chile', 8: 'China', 9: 'Columbia', 10: 'Denmark', 11: 'Dublin', 12: 'Egypt', 13: 'Ethiopia', 14: 'France', 15: 'Germany', 16: 'Greece', 17: 'India'}


In [12]:
dest_country_mapping = dict(enumerate(df['dest_country'].cat.categories))
print(dest_country_mapping)

{0: 'Algeria', 1: 'Argentina', 2: 'Australia', 3: 'Austria', 4: 'Belgium', 5: 'Brazil', 6: 'Canada', 7: 'Chile', 8: 'China', 9: 'Columbia', 10: 'Denmark', 11: 'Dublin', 12: 'Egypt', 13: 'Ethiopia', 14: 'France', 15: 'Germany', 16: 'Greece', 17: 'India', 18: 'Indonesia', 19: 'Italy', 20: 'Japan', 21: 'Kenya', 22: 'Malaysia', 23: 'Mexico', 24: 'Morocco', 25: 'Netherlands', 26: 'Norway', 27: 'Panama', 28: 'Peru', 29: 'Philippines', 30: 'Portugal', 31: 'Qatar', 32: 'Rome', 33: 'Russia', 34: 'Singapore', 35: 'South Africa', 36: 'South Korea', 37: 'Spain', 38: 'Sweden', 39: 'Taiwan', 40: 'Thailand', 41: 'Turkey', 42: 'United Arab Emirates', 43: 'United Kingdom', 44: 'United States', 45: 'Vietnam', 46: 'Zurich'}


In [13]:
airline_name_mapping = dict(enumerate(df['airline_name'].cat.categories))
print(airline_name_mapping)

{0: 'ANA', 1: 'ASL Airlines', 2: 'Aegean', 3: 'Aer Lingus', 4: 'Aerolineas Argentinas', 5: 'Aeromexico', 6: 'Air Algerie', 7: 'Air Arabia', 8: 'Air Arabia Maroc', 9: 'Air Astana', 10: 'Air Baltic', 11: 'Air Canada', 12: 'Air China', 13: 'Air Dolomiti', 14: 'Air Europa', 15: 'Air France', 16: 'Air India', 17: 'Air Macau', 18: 'Air Malta', 19: 'Air Moldova', 20: 'Air New Zealand', 21: 'Air Serbia', 22: 'Air Tahiti Nui', 23: 'Air Transat', 24: 'AirAsia (India)', 25: 'AirAsia X', 26: 'Aircalin', 27: 'American', 28: 'Arkia', 29: 'Asiana', 30: 'Austrian', 31: 'Avianca', 32: 'Azores Airlines', 33: 'Azul', 34: 'Biman', 35: 'BoA', 36: 'British Airways', 37: 'Brussels Airlines', 38: 'Bulgaria Air', 39: 'COPA', 40: 'CSA', 41: 'Cathay Pacific', 42: 'China Airlines', 43: 'China Eastern', 44: 'China Southern', 45: 'Croatia', 46: 'Delta', 47: 'EVA Air', 48: 'EgyptAir', 49: 'El Al', 50: 'Emirates', 51: 'Ethiopian', 52: 'Etihad', 53: 'Eurowings', 54: 'Eurowings Discover', 55: 'Fiji Airways', 56: 'Finna

In [14]:
# Convert text data to numbers

df['from_country'] = df['from_country'].cat.codes
df['dest_country'] = df['dest_country'].cat.codes
df['airline_name'] = df['airline_name'].cat.codes

In [15]:
# Define the X columns and Y columns

X = df[['from_country','dest_country','airline_name','duration','stops','co2_emissions','departure_month']]
Y = df[['price']]

In [16]:
# Standardization

sc = StandardScaler()
X = sc.fit_transform(X)

In [17]:
# Split the dataset into training data and testing data

Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [18]:
model_names = []
mse_scores = []
mae_scores = []
r2_scores = []

## Decision Tree Model

In [70]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

DT = DecisionTreeRegressor()

param_grid = {'max_depth': range(1, 10)}

# Cross validation
grid_search = GridSearchCV(DT, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(Xtrain, Ytrain)

print("Best hyperparameters:", grid_search.best_params_)

Best hyperparameters: {'max_depth': 9}


In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

# Decision Tree Model

DTR = DecisionTreeRegressor(max_depth = 9).fit(Xtrain, Ytrain)
Ypred_dtr = DTR.predict(Xtest)

In [21]:
# Evaluate the model's performance
r_squared_dtr = r2_score(Ytest, Ypred_dtr)
mse_dtr = mean_squared_error(Ytest, Ypred_dtr)
mae_dtr = mean_absolute_error(Ytest, Ypred_dtr)

# Print the evaluation metrics
print(f"R-squared: {r_squared_dtr:.2f}")
print(f"Mean squared error: {mse_dtr:.2f}")
print(f"Mean absolute error: {mae_dtr:.2f}")

# Write in the matrix
model_names.append('Decision Tree')
mse_scores.append(mse_dtr)
mae_scores.append(mae_dtr)
r2_scores.append(r_squared_dtr)

R-squared: 0.65
Mean squared error: 1275969.58
Mean absolute error: 668.89


In [22]:
# Overfitting Check

Ytrain_test_dtr = DTR.predict(Xtrain)
r_squared_train_dtr = r2_score(Ytrain, Ytrain_test_dtr)
print(f"R-squared: {r_squared_train_dtr:.2f}")

R-squared: 0.72


## Random Forest

In [76]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor()

param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': range(1, 10),
    'min_samples_split': range(1,10)
}

# Cross Validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(Xtrain,Ytrain)

print(grid_search.best_params_)

Fitting 5 folds for each of 243 candidates, totalling 1215 fits


/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

...

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:598: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selec

/Users/nannmemeda/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:880: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'max_depth': 9, 'min_samples_split': 8, 'n_estimators': 100}


In [23]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Random Forest Model

RF = RandomForestRegressor(max_depth=9, min_samples_split=8, n_estimators=100).fit(Xtrain, Ytrain)
Ypred_rf = RF.predict(Xtest)

/var/folders/t3/ctmyp9cd5j7ctjhqp2_g843h0000gn/T/ipykernel_56736/4083413867.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RF = RandomForestRegressor(max_depth=9, min_samples_split=8, n_estimators=100).fit(Xtrain, Ytrain)


In [24]:
# Evaluate the model's performance
r_squared_rf = r2_score(Ytest, Ypred_rf)
mse_rf = mean_squared_error(Ytest, Ypred_rf)
mae_rf = mean_absolute_error(Ytest, Ypred_rf)

# Print the evaluation metrics
print(f"R-squared: {r_squared_rf:.2f}")
print(f"Mean squared error: {mse_rf:.2f}")
print(f"Mean absolute error: {mae_rf:.2f}")

# Write in the matrix
model_names.append('Random Forest')
mse_scores.append(mse_rf)
mae_scores.append(mae_rf)
r2_scores.append(r_squared_rf)

R-squared: 0.69
Mean squared error: 1129709.10
Mean absolute error: 648.41


In [25]:
# Overfitting Check

Ytrain_test_rf = RF.predict(Xtrain)
r_squared_train_rf = r2_score(Ytrain, Ytrain_test_rf)
print(f"R-squared: {r_squared_train_rf:.2f}")

R-squared: 0.73


In [40]:
dump(RF, 'RF.joblib')

['RF.joblib']

## Linear Regression Model

In [26]:
from sklearn.linear_model import LinearRegression

# Linear Regression Model

LR = LinearRegression().fit(Xtrain, Ytrain)
Ypred_lr = LR.predict(Xtest)

In [27]:
# Evaluate the model's performance
r_squared_lr = r2_score(Ytest, Ypred_lr)
mse_lr = mean_squared_error(Ytest, Ypred_lr)
mae_lr = mean_absolute_error(Ytest, Ypred_lr)

# Print the evaluation metrics
print(f"R-squared: {r_squared_lr:.2f}")
print(f"Mean squared error: {mse_lr:.2f}")
print(f"Mean absolute error: {mae_lr:.2f}")

# Write in the matrix
model_names.append('Linear Regression')
mse_scores.append(mse_lr)
mae_scores.append(mae_lr)
r2_scores.append(r_squared_lr)

R-squared: 0.53
Mean squared error: 1680029.12
Mean absolute error: 806.78


In [28]:
# Overfitting Check

Ytrain_test_lr = LR.predict(Xtrain)
r_squared_train_lr = r2_score(Ytrain, Ytrain_test_lr)
print(f"R-squared: {r_squared_train_lr:.2f}")

R-squared: 0.51


## KNN

In [32]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()

param_grid = {'n_neighbors': range(2,10)}

# Cross Validation
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(Xtrain, Ytrain)

best_params = grid_search.best_params_
print('Best hyperparameters:', best_params)

Best hyperparameters: {'n_neighbors': 5}


In [33]:
from sklearn.neighbors import KNeighborsRegressor

# KNN Regression Model

KN = KNeighborsRegressor(n_neighbors=5).fit(Xtrain, Ytrain)
Ypred_kn = KN.predict(Xtest)

In [34]:
# Evaluate the model's performance
r_squared_kn = r2_score(Ytest, Ypred_kn)
mse_kn = mean_squared_error(Ytest, Ypred_kn)
mae_kn = mean_absolute_error(Ytest, Ypred_kn)

# Print the evaluation metrics
print(f"R-squared: {r_squared_kn:.2f}")
print(f"Mean squared error: {mse_kn:.2f}")
print(f"Mean absolute error: {mae_kn:.2f}")

# Write in the matrix
model_names.append('KNN Regression')
mse_scores.append(mse_kn)
mae_scores.append(mae_kn)
r2_scores.append(r_squared_kn)

R-squared: 0.65
Mean squared error: 1271509.31
Mean absolute error: 637.40


In [41]:
# Overfitting Check

Ytrain_test_kn = KN.predict(Xtrain)
r_squared_train_kn = r2_score(Ytrain, Ytrain_test_kn)
print(f"R-squared: {r_squared_train_kn:.2f}")

R-squared: 0.75


# Model Comparison

In [38]:
model_check = pd.DataFrame({
    "Model": model_names,
    "MSE": mse_scores,
    "MAE": mae_scores,
    "R-squared": r2_scores
})

In [39]:
model_check

,Model,MSE,MAE,R-squared
0,Decision Tree,1.275970e+06,668.893155,0.645873
1,Random Forest,1.129709e+06,648.409261,0.686466
2,Linear Regression,1.680029e+06,806.775685,0.533733
3,KNN Regression,1.271509e+06,637.396980,0.647111


Because Random Forest has the lowest MSE and low MAE, and it has the highest R-squared, we chose this model as our final prediction model